In [4]:
import numpy as np
import pandas as pd

pd_data = pd.read_csv('전국평생학습강좌표준데이터.csv',encoding='cp949')
drop_list=['교육시작시각','교육종료시각','강좌내용','운영기관전화번호']
pd_data.drop(drop_list,axis = 1,inplace = True)
# for e in enumerate(pd_data.columns):
#     print(e)
print(pd_data.dtypes)

강좌명                object
강사명                object
교육시작일자             object
교육종료일자             object
교육대상구분             object
교육방법구분             object
운영요일               object
교육장소               object
강좌정원수               int64
수강료                 int64
교육장도로명주소           object
운영기관명              object
접수시작일자             object
접수종료일자             object
접수방법구분             object
선정방법구분             object
홈페이지주소             object
직업능력개발훈련비지원강좌여부    object
학점은행제평가(학점)인정여부    object
평생학습계좌제평가인정여부      object
데이터기준일자            object
제공기관코드             object
제공기관명              object
dtype: object


In [17]:
for col_name in pd_data.columns:
    if '일자' in col_name:
        pd_data[col_name]=pd_data[col_name].astype(np.datetime64,copy = False)
    elif ('구분' in col_name) or ('여부' in col_name) or ('코드' in col_name):
        pd_data[col_name] = pd_data[col_name].astype('category')
        
print(pd_data.dtypes)

강좌명                        object
강사명                        object
교육시작일자             datetime64[ns]
교육종료일자             datetime64[ns]
교육대상구분                   category
교육방법구분                   category
운영요일                       object
교육장소                       object
강좌정원수                       int64
수강료                         int64
교육장도로명주소                   object
운영기관명                      object
접수시작일자             datetime64[ns]
접수종료일자             datetime64[ns]
접수방법구분                   category
선정방법구분                   category
홈페이지주소                     object
직업능력개발훈련비지원강좌여부          category
학점은행제평가(학점)인정여부          category
평생학습계좌제평가인정여부            category
데이터기준일자            datetime64[ns]
제공기관코드                   category
제공기관명                      object
dtype: object


C:\Users\ITPS\AppData\Local\Temp\ipykernel_9820\4045533042.py:3: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  pd_data[col_name]=pd_data[col_name].astype(np.datetime64,copy = False)
C:\Users\ITPS\AppData\Local\Temp\ipykernel_9820\4045533042.py:3: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  pd_data[col_name]=pd_data[col_name].astype(np.datetime64,copy = False)
C:\Users\ITPS\AppData\Local\Temp\ipykernel_9820\4045533042.py:3: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  pd_data[col_name]=pd_data[col_name].astype(np.datetime64,copy = False)


In [30]:
t = []
for col_name in pd_data.columns:
    if pd_data[col_name].hasnans == True:
       # print(col_name)
        cnt_s = pd_data[col_name].isna().value_counts()
        cnt = cnt_s.to_numpy()
        t.append([col_name,cnt[0],cnt[1]])
    #print(pd_data[col_name].isna().value_counts)
#print(t)
t = np.array(t)
missing_df = pd.DataFrame(t[:,1:],index=t[:,0],columns=['value_count','NaN_count'])
#missing_df.index = missing_df['col_name']
missing_df

,value_count,NaN_count
운영요일,22007,4528
교육장소,23755,2780
접수시작일자,21516,5019
접수종료일자,21474,5061
선정방법구분,22273,4262
홈페이지주소,24221,2314


In [33]:
f1 = pd_data['운영요일'].isna()
#pd_data['운영요일'][f1]
pd_data['운영요일'].value_counts()

# 운영요일별 카운트시 운영요일을 특정할 수 없으므로,
# 삭제 조치하도록 함


목          2556
화          2524
수          2434
월          1937
금          1851
           ... 
월+화+토         1
수요일+일요일       1
월요일+일요일       1
격주토요일         1
월 +수 +금       1
Name: 운영요일, Length: 218, dtype: int64

In [53]:
pd_data['교육장소'].value_counts()

# 교육방법 구분이 온라인으로 되어있을 경우, 온라인으로 결치를 대체
# 그렇지 않을경우, 삭제.
pd_data['교육방법구분'].value_counts()
filter1 = pd_data['교육장소'].isna()
#print(pd_data['교육방법구분'][filter1].value_counts())

filter2 = pd_data['교육방법구분'] == '온라인'
pd_data['교육장소'][filter2] = '온라인'
pd_data['교육장소'].isna().value_counts()

# 접수시작/종료일자 결측치
# 교육방법 구분이 온라인을 되어있을 경우 1월1일 부터 12월 31까지의 값으로 대체
# 그렇지 않을경우, 삭제


C:\Users\ITPS\AppData\Local\Temp\ipykernel_9820\676745541.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_data['교육장소'][filter2] = '온라인'


False    24706
True      1829
Name: 교육장소, dtype: int64

In [54]:
for i in range(pd_data.shape[0]):
    row = pd_data.iloc[i,:]
    if (row['교육장소'] == np.NaN) and (row['교육방법구분']=='온라인'):
        row['교육장소'] = '온라인'


In [35]:
pd_data['선정방법구분'].value_counts()
# 결측치 삭제

선착순         16861
추첨           1219
제한없음          516
추첨+선착순        451
인터넷 추첨        392
            ...  
방문접수후 면접        1
방문심사            1
읍면 추천           1
군부대별 선정         1
자체심사            1
Name: 선정방법구분, Length: 72, dtype: int64